In [20]:
import numpy as np
from sklearn.cluster import KMeans

In [21]:
class clusterGraph:
    def __init__(self,k,graph, sigma):
        self.k = k
        self.graph = graph
        self.sigma = sigma

    def affMatrix(self):
        n = self.graph.shape[0]
        A = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    A[i,j] = np.linalg.norm(self.graph[i]-self.graph[j]) **2
                else:
                    A[i,j] = 0
        A = np.exp(-A / (2 * self.sigma**2))
        np.fill_diagonal(A, 0)
        return A
    
    def LMatrix(self, affMatrix):
        n = affMatrix.shape[0]
            
        degrees = np.sum(affMatrix, axis=1)
        sqrt_degrees = np.sqrt(degrees)
        D = np.zeros((n, n))
        
        for i in range(n):
            if sqrt_degrees[i] > 0:
                D[i, i] = 1.0 / sqrt_degrees[i]
            else:
                D[i, i] = 0
                
        #  L = D^(-1/2) * A * D^(-1/2)
        L = D @ affMatrix @ D
        return L

    def getEigenvectors(self, Lmatrix):
        eigenvalues,eigenvectors = np.linalg.eigh(Lmatrix)
        sorted_indices = np.argsort(eigenvalues)[::-1]
        first_k_vectors = sorted_indices[:-self.k]
        X = eigenvectors[:, first_k_vectors]
        
        return X

    def formY(self, X):
            # Étape 4 : Normalisation
            
            # np.linalg.norm remplace : sqrt( sum( X**2 ) )
            # axis=1 : ligne par ligne
            # keepdims=True : (n, 1) 
            norms = np.linalg.norm(X, axis=1, keepdims=True)
            norms[norms == 0] = 1
            #puis on divise
            Y = X / norms
            
            return Y
    def finalClustering(self, Y):
            # Étape 5 et 6 
            
            #  K-Means pour trouver 'k' groupes (self.k)
            kmeans = KMeans(n_clusters=self.k, random_state=42)
            labels = kmeans.fit_predict(Y)
            
            return labels
    
    def cluster(self):
        A = self.affMatrix()
        L = self.LMatrix(A)
        X = self.getEigenvectors(L)
        Y = self.formY(X)
        return self.finalClustering(Y)




In [22]:
k = 2
sigma = 1
# data = np.loadtxt("example1.dat", delimiter=",")
# print(data.shape)
# print(type(data))
data =np.array( [[1,2],[1,3],[2,3],[2,4],[4,5],[5,6],[6,4]] )
model = clusterGraph(k=k, graph=data, sigma=sigma)
labels = model.cluster()

# 4. Affichage des résultats
print("\nRésultat du clustering (Labels) :")
print(labels)


Résultat du clustering (Labels) :
[0 1 0 0 1 1 1]
